# Automatic Data Cost Updating for Cloud Data Benchmarks Project

## Imports 

In [1]:
import gcsfs
import pandas as pd
import pygsheets as pg
import os

## Google Inputs

In [2]:
projID = 'modular-magpie-167320'
bucket = 'gs://cloud-data-benchmarks'

mr_storage = 0.026 # Multi-Region Standard Storage Cost (GB/month)
stor_obj_get = 0.004 # storage.objects.get Cost (/10,000 operations)

# Egress Costs
egress1 = 0.12 # 0-1 TB
egress2 = 0.11 # 1-10 TB
egress3 = 0.08 # 10+ TB
egress = [egress1, egress2, egress3]

# Access Variables
users = 10
executions_per_user = 5

## Cost Analysis

In [3]:
class bucketCost():
    def __init__(self):
        global projID, bucket, egress, stor_obj_get, mr_storage, users, executions_per_user
        self.gs = gcsfs.GCSFileSystem(project= projID)
        self.bucket = bucket
        self.egress = egress
        self.stor_cost = mr_storage
        self.op_cost = (stor_obj_get / 10000) # Add more operation costs as needed
        self.users = users
        self.executions = executions_per_user
        self.names = []
        self.size = []
        self.total_opcost = []
        self.total_objects = []
        self.df = pd.DataFrame()
        
    def fileNames(self):
        for i in self.gs.ls(self.bucket):
            self.names.append(i.split('/')[-1])
        self.df['File Name'] = pd.DataFrame(self.names)
        
    def sizes(self):
        self.index = len(self.df)
        for i in self.gs.ls(self.bucket):
            temp = self.gs.du(i)
            self.size.append(temp)
        self.df['Size (bytes)'] = pd.DataFrame(self.size)
        
    def objectCounter(self, path):
        objects = 0
        for i in self.gs.expand_path(path, 'recursive'):
            if self.gs.info(i)['size'] == 0:
                pass
            else:
                objects += 1
        return objects
        
    def operationCost(self): # This operation cost is for operations that have a /object price
        for i in self.gs.ls(self.bucket):
            if self.gs.info(i)['size'] == 0:
                objects = self.objectCounter(i)
            else:
                objects = 1
            self.total_opcost.append(objects * self.op_cost * self.users * self.executions)
            self.total_objects.append(objects)
        self.df['Number of Objects'] = pd.DataFrame(self.total_objects)
        self.df['GCS Operation Cost'] = pd.DataFrame(self.total_opcost)
        
    def storageCost(self):
        self.df['GCS Storage Cost'] = (self.df['Size (bytes)']/1e9) * self.stor_cost

    def egressCost(self):
        total_size = self.df['Size (bytes)'].sum()
        if total_size > 0 & total_size < 1e12:
            self.egress_cost = self.egress[0] * self.users * self.executions * (total_size/(10**9))
        elif total_size <= 0:
            print('No files in bucket!')
        elif total_size >= 1e12 & total_size < 10e12:
            self.egress_cost = self.egress[1] * self.users * self.executions * (total_size/(10**9))
        else:
            self.egress_cost = self.egress[2] * self.users * self.executions * (total_size/(10**9))
            
    def totalCosts(self):
        self.fileNames()
        self.sizes()
        self.operationCost()
        self.storageCost()
        self.egressCost()
        self.df.loc[len(self.df.index)-1] = self.df.sum()
        self.df.loc[len(self.df.index)-1, 'File Name'] = 'Totals'
        return self.df, self.egress_cost
                
bucketCost = bucketCost()    

In [4]:
df, egress_cost = bucketCost.totalCosts()
bucketCost = egress_cost + df.loc[len(df.index)-1, 'GCS Operation Cost':'GCS Storage Cost'].sum()
df2 = pd.DataFrame({'Cost Type':['GCS Egress Cost', 'GCS Total Cost'], 'Amount ($/month)':[egress_cost, bucketCost]})

## AWS Costs 

In [5]:
total_bytes = df['Size (bytes)'][df.index[-1]]
aws_egress_cost = 0.09 # /GB/month
aws_op_cost = []
aws_storage_cost = []

if total_bytes > 0 & total_bytes <= int(50*10**12):
    east1_storage = 0.023
elif total_bytes > int(50*10**12) & total_bytes <= (450*10**12):
    east1_storage = 0.022
elif total_bytes <= 0:
    print('Your bucket is either empty or you have done your math incorrectly.')
else:
    east1_storage = 0.021
    
stor_obj_get_aws = 0.0004 # Per 10,000

for i in range(len(df.index)):
    bytes = df['Size (bytes)'][i]
    aws_storage_cost.append((bytes/(10**9)) * east1_storage)
    objects = df['Number of Objects'][i]
    aws_op_cost.append((stor_obj_get_aws/1000)*executions_per_user*users*objects)
df['S3 Operation Cost'] = pd.DataFrame(aws_op_cost)
df['S3 Storage Cost'] = pd.DataFrame(aws_storage_cost)
aws_total_egress = (total_bytes/(10**9))*aws_egress_cost*executions_per_user*users

df2.loc[len(df2.index)] = ['AWS Egress Cost',aws_total_egress]
df2.loc[len(df2.index)] = ['AWS Total Cost', aws_total_egress + df.loc[len(df.index)-1, 'S3 Operation Cost':'S3 Storage Cost'].sum()
]

In [7]:
df

,File Name,Size (bytes),Number of Objects,GCS Operation Cost,GCS Storage Cost,S3 Operation Cost,S3 Storage Cost
0,ETOPO1_Ice_g_gmt4.100MB.partparquet,1773177201,52,0.00104,4.610261e-02,0.00104,4.078308e-02
1,ETOPO1_Ice_g_gmt4.500MB.partparquet,1760491231,12,0.00024,4.577277e-02,0.00024,4.049130e-02
2,ETOPO1_Ice_g_gmt4.500MB.partparquet2,1760491231,12,0.00024,4.577277e-02,0.00024,4.049130e-02
3,ETOPO1_Ice_g_gmt4.50MB.partcsv,8909445306,104,0.00208,2.316456e-01,0.00208,2.049172e-01
4,ETOPO1_Ice_g_gmt4.50MB.partparquet,1792339506,104,0.00208,4.660083e-02,0.00208,4.122381e-02
5,ETOPO1_Ice_g_gmt4.csv,6676258865,1,0.00002,1.735827e-01,0.00002,1.535540e-01
6,ETOPO1_Ice_g_gmt4.gzip.partparquet,1154071862,52,0.00104,3.000587e-02,0.00104,2.654365e-02
7,ETOPO1_Ice_g_gmt4.lz4.partparquet,1772640044,52,0.00104,4.608864e-02,0.00104,4.077072e-02
8,ETOPO1_Ice_g_gmt4.lz4.partparquet2,1772640044,52,0.00104,4.608864e-02,0.00104,4.077072e-02
9,ETOPO1_Ice_g_gmt4.nc,362236855,1,0.00002,9.418158e-03,0.00002,8.331448e-03


In [8]:
df2

,Cost Type,Amount ($/month)
0,GCS Egress Cost,943.454719
1,GCS Total Cost,947.605323
2,AWS Egress Cost,707.591040
3,AWS Total Cost,711.269916


## Send to Google Sheet 

In [ ]:
gd = pg.authorize('/home/ubuntu/cloud-data-files/client_secret.json')
key = '12dJHsComJTc-8RbbAzMIVeu21KsxAOediYOpzzUKNAg'
sh = gd.open_by_key(key)
wks = sh[0]
wks.clear('A2:E' + str(len(df.index)+2))
wks.update_value('A1', "All Costs are in $/month")
wks.set_dataframe(df, 'A2')
wks.set_dataframe(df2, 'F2')